In [ ]:
!pip install ../input/efficientnet/efficientnet-1.0.0-py3-none-any.whl

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("../input/deepfake-detection-challenge/sample_submission.csv")

# %% [code]
import tensorflow as tf
import cv2
detection_graph = tf.Graph()
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile('../input/mobilenet-face/frozen_inference_graph_face.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess=tf.compat.v1.Session(graph=detection_graph, config=config)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    boxes_tensor = detection_graph.get_tensor_by_name('detection_boxes:0')    
    scores_tensor = detection_graph.get_tensor_by_name('detection_scores:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
def get_img(images):
    global boxes,scores,num_detections
    im_heights,im_widths=[],[]
    imgs=[]
    for image in images:
        (im_height,im_width)=image.shape[:-1]
        #image_np=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        imgs.append(image)
        im_heights.append(im_height)
        im_widths.append(im_widths)
    imgs=np.array(imgs)
    (boxes, scores_) = sess.run(
        [boxes_tensor, scores_tensor],
        feed_dict={image_tensor: imgs})
    finals=[]
    #print(num_detections)
    for x in range(boxes.shape[0]):
        scores=scores_[x]
        max_=np.where(scores==scores.max())[0][0]
        box=boxes[x][max_]
        ymin, xmin, ymax, xmax = box
        (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                      ymin * im_height, ymax * im_height)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        finals.append([left, right, top, bottom])
    return finals
def resize_imgs(bbox,imgs,resize=240,pad=40):
    finals=[]
    for image,(left, right, top, bottom) in zip(imgs,bbox):
        finals.append(cv2.cvtColor(cv2.resize(image[max([0,top-pad]):bottom+(pad*2),max([0,left-pad]):right+(pad*2)],(resize,resize)),cv2.COLOR_BGR2RGB))
    return np.hstack(finals)
def detect_video(video,frame_count=10):
    capture = cv2.VideoCapture(video)
    v_len = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0,v_len,frame_count, endpoint=False, dtype=np.int)
    imgs=[]
    i=0
    for frame_idx in range(int(v_len)):
        ret = capture.grab()
        if not ret: 
            print("Error grabbing frame %d from movie %s" % (frame_idx, video))
        if frame_idx >= frame_idxs[i]:
            if frame_idx-frame_idxs[i]>20:
                return None
            ret, frame = capture.retrieve()
            if not ret or frame is None:
                print("Error retrieving frame %d from movie %s" % (frame_idx, video))
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                imgs.append(frame)
            i += 1
            if i >= len(frame_idxs):
                break
    bbox=get_img(imgs)
    if len(bbox)<10:
        return None
    return imgs,bbox

In [ ]:
import torch
import glob
from tqdm.notebook import tqdm
import numpy as np
import os
import torch
from albumentations.pytorch import ToTensorV2
from albumentations import *
val_transform = Compose([
    Resize(240, 240),
    Normalize(mean=(.5, .5, .5), std=(.5, .5, .5)),
    ToTensorV2(),
])

In [ ]:
model1=torch.jit.load('../input/dfdc-ensemble-final-2/dfdc_mixl_0.pt',map_location='cpu').cuda()
_=model1.eval()
model2=torch.jit.load('../input/dfdc-ensemble-final-2/dfdc_mixxl_0.pt',map_location='cpu').cuda()
_=model2.eval()
model3=torch.jit.load('../input/dfdc-ensemble-final-2/dfdc_se101_2.pt',map_location='cpu').cuda()
_=model3.eval()

In [ ]:
import efficientnet.keras as efn 
from keras.layers import *
from keras.models import *
bottleneck = efn.EfficientNetB6(weights=None,include_top=False,pooling='avg')
inp=Input((240,240,3))
x=bottleneck(inp)
x=Dense(256)(x)
x=Dense(128, activation='relu')(x)
x=Dense(1,activation='sigmoid')(x)
model_b6=Model(inp,x)
model_b6.load_weights('../input/dfdc-ensemble-final-2/nocrop_effb6_dense256_dense128_e5_wrong_activation_weights.h5')

In [ ]:
model_34 = load_model("../input/dfdc-ensemble-final-2/model_b3_300_4.h5")
model_35 = load_model("../input/dfdc-ensemble-final-2/model_35.h5")
model_b1_34 = load_model("../input/dfdc-ensemble-final-2/model_b1_200_5.h5")

def get_birghtness(img):
    return img/img.max()
def process_img(img,flip=False,birghtness=True):
    imgs=[]
    for x in range(10):
        if flip:
            imgs.append(get_birghtness(cv2.flip(img[:,x*240:(x+1)*240,:],1)))
        else:
            imgs.append(get_birghtness(img[:,x*240:(x+1)*240,:]))
    return np.array(imgs)

In [ ]:
NUM_SAMPLES=5
def aug(img, size):
    img = A.RandomCrop(size,size)(image=img)['image']
    return img
def aug2(img):
    img = A.HorizontalFlip(p=0.5)(image=img)['image']
    img = A.RandomBrightness(p=0.5)(image=img)['image']
    return img
def process_img2(img, size=200, pad=40):
    imgs=[]
    for x in range(10):
        for _ in range(NUM_SAMPLES):
            imgs.append(aug(img[:,x*size+pad:(x+1)*size+pad,:], size))
    return np.array(imgs)
def process_img3(img):
    imgs=[]
    for x in range(10):
        for _ in range(NUM_SAMPLES):
            imgs.append(aug2(img[:,x*240:(x+1)*240,:]))
    return np.array(imgs)
def torch_process_img(img):
    imgs=[]
    for x in range(10):
        imgs.append(val_transform(image=img[:,x*240:(x+1)*240,:])['image'])
    imgs = torch.stack(imgs)
    return imgs

In [ ]:
import os
import numpy as np
import glob
import albumentations as A
from tqdm.notebook import tqdm
test_files=glob.glob('../input/deepfake-detection-challenge/test_videos/*.mp4')
submission=pd.DataFrame()
submission['filename']=os.listdir(('../input/deepfake-detection-challenge/test_videos/'))
submission['label']=0.5

# %% [code]
filenames=[]
preds=[]
for x in tqdm(test_files):
    try:
        imgs,raw_bbox=detect_video(x)
        video = resize_imgs(raw_bbox, imgs, resize=240,pad=40)
        video2 = resize_imgs(raw_bbox, imgs, resize=340, pad=40)
        img=process_img2(cv2.cvtColor(video,cv2.COLOR_BGR2RGB))
        img2=process_img2(video2, size=300, pad=40)
        img3=process_img2(video)
        img4=process_img3(video)
        img5=torch_process_img(cv2.cvtColor(video,cv2.COLOR_BGR2RGB)).cuda()
        pred1 = model_35.predict(img)
        pred2 = model_34.predict(img2)
        pred3 = model_b1_34.predict(img3)
        pred4 = model_b6.predict(img4)

        p = np.concatenate([pred1, pred2, pred3,pred4])

        #HERE IS COMES, THIS TIME IT IS NOT ABOUT JUST HARD MEAN, ITS ABOUT LOSS CONTROL

        l0 = []
        l1 = []
        for pr in p:
            if pr>0.5:
                l1.append(pr)
            else:
                l0.append(pr)

        if len(l0) is 0: l0 = [0]
        if len(l1) is 0: l1 = [0]
    
        confidence_0 = (0.5 - np.mean(l0)) * 2
        confidence_1 = (np.mean(l1) - 0.5) * 2

        #FORMULA TO SUCCESS meaning with confidence
        p = np.mean(p) - (np.mean(p) / 100) * (confidence_0 * 100) + (np.mean(p) / 100) * (confidence_1 * 100)

        #BUT AFTER THIS, THE POSSIBILITES OF THE SCORES ARE BEYOND 0 AND 1 SO I AM GONNA CLIP TO MAKE SURE EVERYTHING IS IN PLACE

        p1 = np.clip(p, a_max=0.99, a_min=0.01)

        
        pred1=torch.sigmoid(model1(img5)).view(10).mean().item()
        pred2=torch.sigmoid(model2(img5)).view(10).mean().item()
        pred3=torch.sigmoid(model3(img5)).view(10).mean().item()
        p2=np.array([pred1,pred2,pred3]).mean()
        preds.append(np.mean([p1,p2]))
        filenames.append(x.replace('../input/deepfake-detection-challenge/test_videos/',''))
    except Exception as err:
        print(err)
        pass

In [ ]:
np.mean([p1,p2])

In [ ]:
print(sum(preds)/len(preds))
# %% [code]
for x,y in zip(preds,filenames):
    submission.loc[submission['filename']==y,'label']=x

In [ ]:
np.array(submission['label']).mean()

In [ ]:
submission.head()

In [ ]:
# %% [code]
submission.to_csv('submission.csv', index=False)
!rm -r videos